In [9]:
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np

In [10]:
classes = ['kitakata', 'sano']

batch_size = 32
nb_classes = len(classes)

img_rows, img_cols = 150, 150
channels = 3

train_data_dir = 'train_images'
validation_data_dir = 'test_images'

nb_train_samples = 500
nb_val_samples = 50
nb_epoch = 30

steps_per_epoch =  int(nb_train_samples/batch_size)

In [11]:
result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [12]:
# VGG16モデルと学習済み重みをロード
# Fully-connected層（FC）はいらないのでinclude_top=False）
input_tensor = Input(shape=(img_rows, img_cols, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

In [13]:
# 全結合層を構築
# Flattenへの入力指定はバッチ数を除く
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(nb_classes, activation='softmax'))

In [14]:
# VGG16と全結合層を接続
model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

# 最後のconv層の直前までの層は、学習済のため、学習させない
for layer in model.layers[:15]:
    layer.trainable = False

optimizer_adam = optimizers.Adam(lr=1e-4)
optimizer_sgd = optimizers.SGD(lr=1e-4, momentum=0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer_adam,
              metrics=['accuracy'])

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

 # サブディレクトリをクラス名にしておくと、テスト用の画像を生成してくれる
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

Found 998 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [16]:
# Fine-tuning
history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=nb_epoch,
    validation_data=validation_generator,
    validation_steps=nb_val_samples)

Epoch 1/30
15/15 [==============================] - 227s 15s/step - loss: 0.8144 - acc: 0.5271 - val_loss: 0.6698 - val_acc: 0.5063
Epoch 2/30
15/15 [==============================] - 230s 15s/step - loss: 0.6303 - acc: 0.6542 - val_loss: 0.5659 - val_acc: 0.7413
Epoch 3/30
15/15 [==============================] - 233s 16s/step - loss: 0.5364 - acc: 0.7182 - val_loss: 0.4942 - val_acc: 0.7366
Epoch 4/30
15/15 [==============================] - 231s 15s/step - loss: 0.5113 - acc: 0.7458 - val_loss: 0.4994 - val_acc: 0.7603
Epoch 5/30
15/15 [==============================] - 234s 16s/step - loss: 0.4991 - acc: 0.7729 - val_loss: 0.6899 - val_acc: 0.6788
Epoch 6/30
15/15 [==============================] - 241s 16s/step - loss: 0.5297 - acc: 0.7281 - val_loss: 0.4643 - val_acc: 0.7911
Epoch 7/30
15/15 [==============================] - 231s 15s/step - loss: 0.4515 - acc: 0.7636 - val_loss: 0.4766 - val_acc: 0.7508
Epoch 8/30
15/15 [==============================] - 238s 16s/step - loss: 0.

In [17]:
model.save_weights(os.path.join(result_dir, 'finetuning.h5'))